<a href="https://colab.research.google.com/github/newfull5/AI-Project/blob/master/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
import numpy as np
import matplotlib.pyplot as plt
  
# 생성자 모델 만둘가
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
  
# 판별자 모델 만들기
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False
  
# 생성자와 판별자 모델을 연결시키는 gan 모델 만들기 
ginput = Input(shape=(100,))
 
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()
  
# 신경망을 실행시키는 함수 만들기
def gan_train(epoch, batch_size, saving_interval):
  (X_train, _), (_, _) = mnist.load_data()  
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
  # 127.5를 빼준 뒤 127.5로 나눠서 -1~1사이의 값으로 바꿈
  X_train = (X_train - 127.5) / 127.5  
  true = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))
 
  for i in range(epoch):
      # 실제 데이터를 판별자에 입력
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      d_loss_real = discriminator.train_on_batch(imgs, true)
 
      # 가상 이미지를 판별자에 입력
      noise = np.random.normal(0, 1, (batch_size, 100))
      gen_imgs = generator.predict(noise)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
 
      # 판별자와 생성자의 오차 계산
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      g_loss = gan.train_on_batch(noise, true)
 
      print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)
 
gan_train(4001, 32, 200)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_5 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________
epoch:0  d_loss:0.7113  g_loss:0.7673
epoch:1  d_loss:0.5142  g_loss:0.4202
epoch:2  d_loss:0.5971  g_loss:0.1797
epoch:3  d_loss:0.5727  g_loss:0.1652
epoch:4  d_loss:0.4996  g_loss:0.2802
epoch:5  d_loss:0.4585  g_loss:0.4918
epoch:6  d_loss:0.4520  g_loss:0.7354
epoch:7  d_loss:0.4554  g_loss:0.8602
epoch:8  d_loss:0.4840  g_loss:0.8283
epoch:9  d_loss:0.4872  g_loss:0.